# Limpieza de Datos dataset: Salary_Survey
## Importar librerias necesarias

In [404]:
import pandas as pd
import numpy as np
import re
import unicodedata

## Cargar el Dataset con la información suministrada

In [ ]:

SHEET_ID = "1IPS5dBSGtwYVbjsfbaMCYIWnOuRmJcbequohNxCyGVw"
GID = "1625408792"

csv_url = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}"
df = pd.read_csv(csv_url)

print(df.shape)
df.head(3)

(28191, 18)


,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White


In [ ]:
df_base=df.copy()

In [444]:
df=df_base.copy()

In [445]:
df.columns


Index(['Timestamp', 'How old are you?', 'What industry do you work in?',
       'Job title',
       'If your job title needs additional context, please clarify here:',
       'What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)',
       'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.',
       'Please indicate the currency',
       'If "Other," please indicate the currency here: ',
       'If your income needs additional context, please provide it here:',
       'What country do you work in?',
       'If you're in the U.S., what state do you work in?',
       'What city do you work in?',
       'How many years of professional work experience do you have overall?',
       

## Renombrar las Columnas:
Debido a que tienen nombres demasiado extensos, es encesario renombrar las variables con palabras claves, para evitar confusiones y eliminiar ruido en los nombres

In [446]:
rename_map = {
    # Fecha
    "Timestamp": "timestamp",
    # Edad
    "How old are you?": "rango_edad",
    # Industria / trabajo
    "What industry do you work in?": "industria",
    "Job title": "nombre_trabajo",
    "If your job title needs additional context, please clarify here:": "info_adicionales_trabajo",
    # Salarios
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "salario_anual",
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "compensacion_monetaria",
    # Moneda
    "Please indicate the currency": "moneda",
    'If "Other," please indicate the currency here: ': "otra_moneda",
    # Contexto ingresos
    "If your income needs additional context, please provide it here:": "info_adicionales_ingresos",
    # Ubicación
    "What country do you work in?": "pais",
    "If you're in the U.S., what state do you work in?": "estado_pais",
    "What city do you work in?": "ciudad",
    # Experiencia
    "How many years of professional work experience do you have overall?": "experiencia_general",
    "How many years of professional work experience do you have in your field?": "experiencia_campo",
    # Educación
    "What is your highest level of education completed?": "nivel_educacion",
    # Demografía
    "What is your gender?": "genero",
    "What is your race? (Choose all that apply.)": "etnia",
}

df = df.rename(columns=rename_map)

In [447]:
df

,timestamp,rango_edad,industria,nombre_trabajo,info_adicionales_trabajo,salario_anual,compensacion_monetaria,moneda,otra_moneda,info_adicionales_ingresos,pais,estado_pais,ciudad,experiencia_general,experiencia_campo,nivel_educacion,genero,etnia
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28186,12/16/2025 18:10:31,18-24,Transport or Logistics,Apprentie Data et IA,appliquée au domaine du transport,19000,0.0,EUR,NaN,NaN,France,NaN,Paris,1 year or less,1 year or less,Master's degree,Woman,Black or African American
28187,1/6/2026 9:35:36,35-44,Engineering or Manufacturing,Principal Engineer,NaN,151000,0.0,USD,NaN,NaN,United States,Iowa,Cedar Rapids,11 - 20 years,11 - 20 years,Master's degree,Woman,White
28188,1/8/2026 16:02:32,55-64,Computing or Tech,Senior Technical Writer,NaN,125000,5000.0,CAD,NaN,NaN,Canada,NaN,Toronto,31 - 40 years,21 - 30 years,College degree,Woman,White
28189,2/2/2026 13:20:26,25-34,Retail,Marketing Manager,Email & SMS,125000,0.0,USD,NaN,NaN,USA,Wisconsin,Eau Claire,8 - 10 years,8 - 10 years,College degree,Woman,White


In [448]:
df.describe()

,compensacion_monetaria
count,2.083500e+04
mean,1.822711e+04
std,8.327849e+05
min,0.000000e+00
25%,0.000000e+00
50%,2.000000e+03
75%,1.000000e+04
max,1.200000e+08


### Información presentada en Describe:
Solo se estan mostrando datos de compensación monetaria en el describe(indica que solo los valores presentados son de tipo numerico) por lo que se comienza a realizar limpieza del dataset

In [449]:
for column in df.columns:
    print(f"Unique values for column '{column}':")
    print(df[column].unique())
    print("\n" + "-" * 50 + "\n")

Unique values for column 'timestamp':
['4/27/2021 11:02:10' '4/27/2021 11:02:22' '4/27/2021 11:02:38' ...
 '1/8/2026 16:02:32' '2/2/2026 13:20:26' '2/8/2026 19:13:25']

--------------------------------------------------

Unique values for column 'rango_edad':
['25-34' '45-54' '35-44' '18-24' '65 or over' '55-64' 'under 18']

--------------------------------------------------

Unique values for column 'industria':
['Education (Higher Education)' 'Computing or Tech'
 'Accounting, Banking & Finance' ... 'School' 'lifeguard' 'study']

--------------------------------------------------

Unique values for column 'nombre_trabajo':
['Research and Instruction Librarian'
 'Change & Internal Communications Manager' 'Marketing Specialist' ...
 'CPD Instructor' 'Apprentie Data et IA ' 'Analytics Performance']

--------------------------------------------------

Unique values for column 'info_adicionales_trabajo':
[nan 'High school, FT' 'Data developer/ETL Developer' ... 'tking a course'
 'appliquée

In [450]:
df.nunique()

,0
timestamp,25409
rango_edad,7
industria,1225
nombre_trabajo,14418
info_adicionales_trabajo,7026
salario_anual,4328
compensacion_monetaria,849
moneda,11
otra_moneda,129
info_adicionales_ingresos,2987


## Revisar valores duplicados, nulos y porcentaje de nulos
Para evitar datos que no son necesarios, y datos que van a generar ruido, se realiza una tabla especificando cada una de las columnas con valores unicos, nulos y porcentaje de los ultimos

In [451]:
perfil = pd.DataFrame({
    "valores_unicos": df.nunique(),
    "nulos": df.isna().sum(),
    "porcentaje_nulos": (df.isna().sum() / len(df)) * 100
})

perfil.sort_values("valores_unicos", ascending=False)

,valores_unicos,nulos,porcentaje_nulos
timestamp,25409,0,0.000000
nombre_trabajo,14418,2,0.007094
info_adicionales_trabajo,7026,20902,74.144230
ciudad,4868,82,0.290873
salario_anual,4328,0,0.000000
info_adicionales_ingresos,2987,25140,89.177397
industria,1225,81,0.287326
compensacion_monetaria,849,7356,26.093434
pais,390,0,0.000000
estado_pais,137,5063,17.959633


### **Eliminar del DF columnas no necesarias**
#### **TimeStamp**: Variable no relevante
#### **info_adicionales_trabajo**:  es una variable que no es parametrizable facilmente, ademas para la información que va a ser usada en el dashboard no generan impacto, sería necesario ver los datos de cada una de las personas.
#### **variables con valores nulos** que no tengan impacto en el % mayor a 1% entre estas variables estan: nombre_trabajo, ciudad, industria, etnia, nivel_educacion, genero.

In [452]:
df = df.drop(columns=["timestamp"])
df = df.drop(columns=["info_adicionales_trabajo"])

cols_valores_nulos = [
    "nombre_trabajo",
    "ciudad",
    "industria",
    "etnia",
    "genero",
    "nivel_educacion"
]


print("Registros iniciales:", df.shape[0])
df = df.dropna(subset=cols_valores_nulos)
print("Registros luego de limpieza:", df.shape[0])

Registros iniciales: 28191
Registros luego de limpieza: 27561


In [453]:
df[cols_valores_nulos].isna().sum()

,0
nombre_trabajo,0
ciudad,0
industria,0
etnia,0
genero,0
nivel_educacion,0


#### Descripcción de la variable salarial

In [454]:
df["salario_anual_num"] = pd.to_numeric(
    df["salario_anual"],
    errors="coerce"
)

df["compensacion_num"] = pd.to_numeric(
    df["compensacion_monetaria"],
    errors="coerce"
)

In [455]:
no_salario = df["salario_anual_num"].isna().sum()
no_comp = df["compensacion_num"].isna().sum()

print("Salarios no convertidos:", no_salario)
print("Compensaciones no convertidas:", no_comp)

Salarios no convertidos: 19902
Compensaciones no convertidas: 7105


In [456]:
df.loc[
    df["salario_anual"].astype(str).str.contains(",", na=False),
    "salario_anual"
].head(10)

,salario_anual
0,"55,000"
1,"54,600"
2,"34,000"
3,"62,000"
4,"60,000"
5,"62,000"
6,"33,000"
7,"50,000"
8,"112,000"
9,"45,000"


In [457]:
df["salario_anual_clean"] = (
    df["salario_anual"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

df["compensacion_clean"] = (
    df["compensacion_monetaria"]
    .astype(str)
    .str.replace(",", "", regex=False)
)

In [458]:
df["salario_anual_num"] = pd.to_numeric(
    df["salario_anual_clean"],
    errors="coerce"
)

df["compensacion_num"] = pd.to_numeric(
    df["compensacion_clean"],
    errors="coerce"
)

In [459]:
print(
    "Salarios no convertidos:",
    df["salario_anual_num"].isna().sum()
)

print(
    "Compensaciones no convertidas:",
    df["compensacion_num"].isna().sum()
)

Salarios no convertidos: 0
Compensaciones no convertidas: 7105


In [460]:
mask_compensacion_num_no_convertido = df["compensacion_num"].isna()

In [461]:
mask_compensacion_num_no_convertido.sum()

np.int64(7105)

In [462]:
df.loc[
    mask_compensacion_num_no_convertido,
    "compensacion_monetaria"
].unique()

array([nan])

In [463]:
df["compensacion_num"] = df["compensacion_num"].fillna(0)

In [464]:
df["compensacion_num"].isna().sum()

np.int64(0)

In [465]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27561 entries, 0 to 28190
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   rango_edad                 27561 non-null  object 
 1   industria                  27561 non-null  object 
 2   nombre_trabajo             27561 non-null  object 
 3   salario_anual              27561 non-null  object 
 4   compensacion_monetaria     20456 non-null  float64
 5   moneda                     27561 non-null  object 
 6   otra_moneda                207 non-null    object 
 7   info_adicionales_ingresos  3008 non-null   object 
 8   pais                       27561 non-null  object 
 9   estado_pais                22669 non-null  object 
 10  ciudad                     27561 non-null  object 
 11  experiencia_general        27561 non-null  object 
 12  experiencia_campo          27561 non-null  object 
 13  nivel_educacion            27561 non-null  object 


In [466]:
df = df.drop(columns=[
    "salario_anual",
    "compensacion_monetaria",
    "salario_anual_clean",
    "compensacion_clean"
])

In [467]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27561 entries, 0 to 28190
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   rango_edad                 27561 non-null  object 
 1   industria                  27561 non-null  object 
 2   nombre_trabajo             27561 non-null  object 
 3   moneda                     27561 non-null  object 
 4   otra_moneda                207 non-null    object 
 5   info_adicionales_ingresos  3008 non-null   object 
 6   pais                       27561 non-null  object 
 7   estado_pais                22669 non-null  object 
 8   ciudad                     27561 non-null  object 
 9   experiencia_general        27561 non-null  object 
 10  experiencia_campo          27561 non-null  object 
 11  nivel_educacion            27561 non-null  object 
 12  genero                     27561 non-null  object 
 13  etnia                      27561 non-null  object 


### Se evidencia que hay varios salarios menos a 1000 por lo que estos pueden ser salarios en miles, posteriormente se va a realizar la multiplicación de estos valores para evitar valores que modifiquen las estadisticas.

In [468]:

df["salario_anual_num"].describe()
df["compensacion_num"].describe()


df["salario_anual_num"].quantile([0.90, 0.95, 0.99])
df["compensacion_num"].quantile([0.90, 0.95, 0.99])

df[df["salario_anual_num"] < 1000]["salario_anual_num"].head(20)

,salario_anual_num
97,58
166,35
895,38
968,61
1607,130
2229,33
4081,35
4124,108
4546,63
5139,118


### Renombrar las Columnas Limpias de salario y compensación,las cuales se encuentran en dolares
Limpiar las variables de moneda, ya que estas son las que van a ayudar a estandarizar los valores a COP
se unen las columnas  moneda y otra moneda.
debido a que other no corresponde a un valor significante en los registros se eliminan estos registros  

In [469]:
df = df.rename(columns={
    "salario_anual_num": "salario_anual_usd",
    "compensacion_num": "compensacion_usd"
})

In [470]:
df


,rango_edad,industria,nombre_trabajo,moneda,otra_moneda,info_adicionales_ingresos,pais,estado_pais,ciudad,experiencia_general,experiencia_campo,nivel_educacion,genero,etnia,salario_anual_usd,compensacion_usd
0,25-34,Education (Higher Education),Research and Instruction Librarian,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White,55000,0.0
1,25-34,Computing or Tech,Change & Internal Communications Manager,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White,54600,4000.0
2,25-34,"Accounting, Banking & Finance",Marketing Specialist,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White,34000,0.0
3,25-34,Nonprofits,Program Manager,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White,62000,3000.0
4,25-34,"Accounting, Banking & Finance",Accounting Manager,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White,60000,7000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28186,18-24,Transport or Logistics,Apprentie Data et IA,EUR,NaN,NaN,France,NaN,Paris,1 year or less,1 year or less,Master's degree,Woman,Black or African American,19000,0.0
28187,35-44,Engineering or Manufacturing,Principal Engineer,USD,NaN,NaN,United States,Iowa,Cedar Rapids,11 - 20 years,11 - 20 years,Master's degree,Woman,White,151000,0.0
28188,55-64,Computing or Tech,Senior Technical Writer,CAD,NaN,NaN,Canada,NaN,Toronto,31 - 40 years,21 - 30 years,College degree,Woman,White,125000,5000.0
28189,25-34,Retail,Marketing Manager,USD,NaN,NaN,USA,Wisconsin,Eau Claire,8 - 10 years,8 - 10 years,College degree,Woman,White,125000,0.0


In [471]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27561 entries, 0 to 28190
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   rango_edad                 27561 non-null  object 
 1   industria                  27561 non-null  object 
 2   nombre_trabajo             27561 non-null  object 
 3   moneda                     27561 non-null  object 
 4   otra_moneda                207 non-null    object 
 5   info_adicionales_ingresos  3008 non-null   object 
 6   pais                       27561 non-null  object 
 7   estado_pais                22669 non-null  object 
 8   ciudad                     27561 non-null  object 
 9   experiencia_general        27561 non-null  object 
 10  experiencia_campo          27561 non-null  object 
 11  nivel_educacion            27561 non-null  object 
 12  genero                     27561 non-null  object 
 13  etnia                      27561 non-null  object 


In [472]:
df["moneda_total"] = df["moneda"].fillna(df["otra_moneda"])
df["moneda_total"] = df.apply(
    lambda x: x["otra_moneda"]
    if x["moneda"] == "Other"
    else x["moneda"],
    axis=1
)

In [473]:
print(" Numero de monedas totales: " ,df["moneda_total"].nunique())
df["moneda_total"].unique()

 Numero de monedas totales:  89


array(['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'INR', 'Peso Argentino',
       'CHF', 'MYR', 'ZAR', 'SEK', 'HKD', 'NOK', 'BR$', 'Dkk', 'TTD',
       'Indian rupees', 'BRL (R$)', 'Mexican pesos', 'CZK', 'DKK',
       'American Dollars', 'Php', 'PLN (Polish zloty)', 'czech crowns',
       'TRY', 'Norwegian kroner (NOK)', 'CNY', 'ILS/NIS', nan,
       'US Dollar', 'BRL', 'NIS (new Israeli shekel)',
       'RMB (chinese yuan)', 'AUD', 'JPY', 'Taiwanese dollars',
       'Philippine Peso', 'SGD', 'KRW (Korean Won)', 'Czk', 'THB', 'IDR ',
       'Sgd', 'Nok', 'NZD', 'DKK ', 'China RMB', 'AUD Australian ', 'LKR',
       'Polish Złoty', 'Philippine peso (PHP)', 'Australian Dollars ',
       'PHP', 'Equity', 'ARS', 'Argentinian peso (ARS)',
       'Israeli Shekels', 'ILS', 'MXN', 'PhP (Philippine Peso)', 'PLN',
       'KRW', 'SAR', 'RM', 'IDR', 'Argentine Peso', 'Philippine Pesos',
       'Ils', 'INR (Indian Rupee)', 'Danish Kroner', 'Korean Won ',
       'Euro', 'SGD ', 'NTD', 'Mexican Pesos', '

In [474]:
mask_completados_moneda = (
    df["moneda"].isna() &
    df["otra_moneda"].notna()
)

In [475]:
df[df["moneda"] == "Other"][
    ["moneda", "otra_moneda"]
].head(20)

,moneda,otra_moneda
434,Other,INR
603,Other,Peso Argentino
1311,Other,MYR
1840,Other,CHF
1924,Other,NOK
2054,Other,NOK
2473,Other,USD
2639,Other,BR$
2980,Other,SEK
3162,Other,Dkk


### MAPEO DE VARIABLE DE MONEDA Y OTRA_MONEDA
 existen diferentes monedas, pero con diferentes variables en mayusculas minusculas escritas de diferente forma, por lo que se mapean los registros existentes para parametrizar.

In [476]:
currency_map = {

    # USD
    "USD": "USD",
    "US Dollar": "USD",
    "American Dollars": "USD",

    # GBP
    "GBP": "GBP",

    # EUR
    "EUR": "EUR",
    "Euro": "EUR",

    # CAD
    "CAD": "CAD",

    # AUD / NZD
    "AUD": "AUD",
    "AUD Australian ": "AUD",
    "Australian Dollars ": "AUD",
    "NZD": "NZD",
    "AUD/NZD": "AUD",
    "AUD Australian": "AUD",
    "Australian Dollars": "AUD",

    # INR
    "INR": "INR",
    "inr": "INR",
    "Indian rupees": "INR",
    "INR (Indian Rupee)": "INR",
    "Rupees": "INR",
    "Rupees ": "INR",

    # BRL
    "BR$": "BRL",
    "BRL": "BRL",
    "BRL (R$)": "BRL",

    # ARS
    "Peso Argentino": "ARS",
    "Argentine Peso": "ARS",
    "Argentinian peso (ARS)": "ARS",
    "ARS": "ARS",

    # MXN
    "Mexican pesos": "MXN",
    "Mexican Pesos": "MXN",
    "MXN": "MXN",

    # DKK
    "Dkk": "DKK",
    "DKK": "DKK",
    "DKK ": "DKK",
    "Danish Kroner": "DKK",

    # NOK
    "NOK": "NOK",
    "Nok": "NOK",
    "Norwegian kroner (NOK)": "NOK",

    # SEK
    "SEK": "SEK",

    # CHF
    "CHF": "CHF",

    # ZAR
    "ZAR": "ZAR",

    # HKD
    "HKD": "HKD",

    # JPY
    "JPY": "JPY",

    # SGD
    "SGD": "SGD",
    "Sgd": "SGD",
    "SGD ": "SGD",
    "Singapore Dollara": "SGD",

    # ILS
    "ILS": "ILS",
    "Ils": "ILS",
    "ILS/NIS": "ILS",
    "NIS (new Israeli shekel)": "ILS",
    "Israeli Shekels": "ILS",

    # PHP
    "Php": "PHP",
    "PHP": "PHP",
    "PhP (Philippine Peso)": "PHP",
    "Philippine Peso": "PHP",
    "Philippine peso (PHP)": "PHP",
    "Philippine Pesos": "PHP",

    # PLN
    "PLN": "PLN",
    "PLN (Polish zloty)": "PLN",
    "Polish Złoty": "PLN",
    "PLN (Zwoty)": "PLN",

    # CZK
    "CZK": "CZK",
    "Czk": "CZK",
    "czech crowns": "CZK",

    # TRY
    "TRY": "TRY",

    # CNY
    "CNY": "CNY",
    "RMB (chinese yuan)": "CNY",
    "China RMB": "CNY",

    # KRW
    "KRW": "KRW",
    "KRW (Korean Won)": "KRW",
    "Korean Won ": "KRW",

    # THB
    "THB": "THB",
    "THAI  BAHT": "THB",
    "Thai Baht": "THB",
    "Thai Baht ": "THB",

    # IDR
    "IDR": "IDR",
    "IDR ": "IDR",

    # SAR
    "SAR": "SAR",

    # COP
    "Pesos Colombianos": "COP",
    "COP": "COP",

    # Otros
    "LKR": "LKR",
    "NGN": "NGN",
    "TZS": "TZS",
    "ZMW": "ZMW",
    "MYR": "MYR",
    "TTD": "TTD",
    "NTD": "TWD",
    "Taiwanese dollars": "TWD",
    "Korean Won": "KRW",
    "RM": "MYR",
    "croatian kuna": "HRK",
}

In [477]:
df["moneda_total_clean"] = (
    df["moneda_total"]
    .str.strip()
    .replace(currency_map)
)

In [478]:
print("Monedas únicas limpias:",
      df["moneda_total_clean"].nunique())

df["moneda_total_clean"].unique()

Monedas únicas limpias: 38


array(['USD', 'GBP', 'CAD', 'EUR', 'AUD', 'INR', 'ARS', 'CHF', 'MYR',
       'ZAR', 'SEK', 'HKD', 'NOK', 'BRL', 'DKK', 'TTD', 'MXN', 'CZK',
       'PHP', 'PLN', 'TRY', 'CNY', 'ILS', nan, 'JPY', 'TWD', 'SGD', 'KRW',
       'THB', 'IDR', 'NZD', 'LKR', 'Equity', 'SAR', 'HRK', 'NGN', 'TZS',
       'COP', 'ZMW'], dtype=object)

In [479]:
df = df[df["moneda_total_clean"] != "Equity"]
df = df[df["moneda_total_clean"] != "ILS"]
df = df[df["moneda_total_clean"].notna()]

In [480]:
print(
    "Monedas únicas limpias:",
    df["moneda_total_clean"].nunique()
)

df["moneda_total_clean"].unique()

Monedas únicas limpias: 36


array(['USD', 'GBP', 'CAD', 'EUR', 'AUD', 'INR', 'ARS', 'CHF', 'MYR',
       'ZAR', 'SEK', 'HKD', 'NOK', 'BRL', 'DKK', 'TTD', 'MXN', 'CZK',
       'PHP', 'PLN', 'TRY', 'CNY', 'JPY', 'TWD', 'SGD', 'KRW', 'THB',
       'IDR', 'NZD', 'LKR', 'SAR', 'HRK', 'NGN', 'TZS', 'COP', 'ZMW'],
      dtype=object)

### OBTENER CONVERSION DE MONEDAS
se obtinen la conversión de cada una de las monedas a USD con el fin de luego usar la TRM para realizar el cambio  a persos colombianos.

In [481]:
fx_to_usd = {

    "USD": 1,
    "GBP": 1.27,
    "EUR": 1.09,
    "CAD": 0.74,
    "AUD": 0.66,
    "NZD": 0.61,
    "CHF": 1.10,
    "SEK": 0.095,
    "NOK": 0.093,
    "DKK": 0.145,

    "MXN": 0.058,
    "BRL": 0.20,
    "ARS": 0.0012,
    "COP": 0.00027,

    "INR": 0.012,
    "MYR": 0.21,
    "SAR": 0.27,

    "CNY": 0.14,
    "JPY": 0.0067,
    "KRW": 0.00075,
    "TWD": 0.032,
    "HKD": 0.128,
    "SGD": 0.74,

    "THB": 0.028,
    "IDR": 0.000064,
    "PHP": 0.018,
    "PLN": 0.25,
    "CZK": 0.044,
    "TRY": 0.031,

    "ZAR": 0.053,
    "NGN": 0.00062,
    "TZS": 0.00040,
    "ZMW": 0.038,
    "LKR": 0.0031,
    "HRK": 0.15,
    "TTD": 0.15
}

In [482]:
# Factor de conversión a USD según moneda
df["fx_to_usd"] = df["moneda_total_clean"].map(fx_to_usd)

# Validaciones
print("Filas totales:", len(df))
print("Filas sin tasa fx_to_usd:", df["fx_to_usd"].isna().sum())

if df["fx_to_usd"].isna().any():
    print("Monedas sin tasa (revisar fx_to_usd):")
    print(df.loc[df["fx_to_usd"].isna(), "moneda_total_clean"].value_counts())

Filas totales: 27550
Filas sin tasa fx_to_usd: 0


In [483]:
df = df.rename(columns={
    "salario_anual_usd": "salario_anual_monto",
    "compensacion_usd": "compensacion_monto"
})

In [484]:
TRM = 3670

df["salario_usd"] = df["salario_anual_monto"] * df["fx_to_usd"]
df["compensacion_usd"] = df["compensacion_monto"] * df["fx_to_usd"]

df["salario_cop"] = df["salario_usd"] * TRM
df["compensacion_cop"] = df["compensacion_usd"] * TRM
df["total_compensacion_cop"] = df["salario_cop"] + df["compensacion_cop"]



In [485]:
df[["moneda_total_clean","fx_to_usd","salario_anual_monto","salario_usd","salario_cop"]].head(10)

,moneda_total_clean,fx_to_usd,salario_anual_monto,salario_usd,salario_cop
0,USD,1.00,55000,55000.0,201850000.0
1,GBP,1.27,54600,69342.0,254485140.0
2,USD,1.00,34000,34000.0,124780000.0
3,USD,1.00,62000,62000.0,227540000.0
4,USD,1.00,60000,60000.0,220200000.0
5,USD,1.00,62000,62000.0,227540000.0
6,USD,1.00,33000,33000.0,121110000.0
7,USD,1.00,50000,50000.0,183500000.0
8,USD,1.00,112000,112000.0,411040000.0
9,USD,1.00,45000,45000.0,165150000.0


#### Validacion de la columna ciudad - estado - pais




In [486]:
print(" Numero de paises totales: " ,df["pais"].nunique())
df["pais"].unique()

print(" Numero de ciudades totales: " ,df["ciudad"].nunique())
df["ciudad"].unique()


print(" Numero de estado_pais totales: " ,df["estado_pais"].nunique())
df["estado_pais"].unique()

 Numero de paises totales:  380
 Numero de ciudades totales:  4783
 Numero de estado_pais totales:  135


array(['Massachusetts', nan, 'Tennessee', 'Wisconsin', 'South Carolina',
       'New Hampshire', 'Arizona', 'Missouri', 'Florida', 'Pennsylvania',
       'Michigan', 'Illinois', 'California', 'Georgia', 'Ohio',
       'District of Columbia', 'Maryland', 'Texas', 'Minnesota',
       'Virginia', 'North Carolina', 'New York', 'New Jersey',
       'Rhode Island', 'Colorado', 'Oregon', 'Washington', 'Indiana',
       'Iowa', 'Nebraska', 'Oklahoma', 'Maine', 'Connecticut',
       'South Dakota', 'West Virginia', 'Idaho', 'Louisiana', 'Montana',
       'Kentucky', 'North Dakota', 'Kansas', 'Vermont', 'Arkansas',
       'Alabama', 'Nevada', 'Delaware', 'New Mexico', 'Hawaii', 'Utah',
       'Mississippi', 'Kentucky, Ohio', 'District of Columbia, Virginia',
       'District of Columbia, Maryland', 'Alaska', 'Arizona, Washington',
       'Georgia, New York', 'California, Colorado', 'California, Oregon',
       'District of Columbia, Maryland, Pennsylvania, Virginia',
       'Arizona, California'

In [487]:


def norm_country_raw(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    x = re.sub(r"\s+", " ", x)
    if x == "":
        return np.nan
    return x

def norm_key(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    x = "".join(c for c in unicodedata.normalize("NFD", x) if unicodedata.category(c) != "Mn")
    x = re.sub(r"\s+", " ", x)
    return x

df["pais_raw"] = df["pais"].apply(norm_country_raw)
df["pais_key"] = df["pais_raw"].apply(norm_key)

#### Patron para eliminar datos duplicados

In [488]:
bad_patterns = [
    r"we don't get raises",
    r"i work for",
    r"i am located",
    r"worldwide",
    r"global",
    r"remote",
    r"currently",
    r"policy",
    r"contracts",
    r"deducted",
    r"commission",
    r"quarterly",
    r"bonus",
    r"salary",
    r"benefits",
    r"country withheld",
    r"ss$|dbfemf|loutreland|uxz",   # basura evidente
    r"^\$",
    r"usd$",                       # 'USD' no es país
    r"^y$",
    r"^na$",
    r"^n/a",
]

bad_regex = re.compile("|".join(bad_patterns), re.IGNORECASE)

df["pais_no_pais"] = (
    df["pais_raw"].astype(str).str.len().gt(35) |
    df["pais_raw"].astype(str).str.contains(bad_regex, na=False) |
    df["pais_raw"].astype(str).str.contains(r"\d", na=False)  # números en país
)

### MAPEOS DE LOS PAISES

In [489]:
country_map = {

    "us": "United States",
    "u.s.": "United States",
    "u.s": "United States",
    "u s": "United States",
    "usa": "United States",
    "u.s.a": "United States",
    "u.s.a.": "United States",
    "the us": "United States",
    "the united states": "United States",
    "united states": "United States",
    "united states ": "United States",
    "united states of america": "United States",
    "united state": "United States",
    "united state of america": "United States",
    "america": "United States",
    "unitedstates": "United States",
    "united  states": "United States",
    "unites states": "United States",
    "uniter statez": "United States",
    "uniteed states": "United States",
    "united statws": "United States",
    "united sates": "United States",
    "united sttes": "United States",
    "united stares": "United States",
    "united statss": "United States",
    "united statees": "United States",
    "united statesp": "United States",
    "united stated": "United States",
    "united stateds": "United States",
    "unitied states": "United States",
    "unted states": "United States",
    "untied states": "United States",
    "us of a": "United States",
    "🇺🇸": "United States",

    "uk": "United Kingdom",
    "u.k.": "United Kingdom",
    "u.k": "United Kingdom",
    "united kingdom": "United Kingdom",
    "england": "United Kingdom",
    "scotland": "United Kingdom",
    "wales": "United Kingdom",
    "northern ireland": "United Kingdom",
    "great britain": "United Kingdom",
    "britain": "United Kingdom",

    "netherlands": "Netherlands",
    "the netherlands": "Netherlands",
    "nederland": "Netherlands",
    "nl": "Netherlands",

    "canada": "Canada",
    "canadw": "Canada",
    "canda": "Canada",
    "csnada": "Canada",
    "canad": "Canada",
    "canadá": "Canada",

    "australia": "Australia",
    "australian": "Australia",
    "new zealand": "New Zealand",
    "aotearoa new zealand": "New Zealand",
    "nz": "New Zealand",

    "czech republic": "Czechia",
    "česká republika": "Czechia",
    "czechia": "Czechia",

    "uae": "United Arab Emirates",
    "united arab emirates": "United Arab Emirates",

    "puerto rico": "Puerto Rico",
    "hong kong": "Hong Kong",
    "taiwan": "Taiwan",

    "france": "France",
    "germany": "Germany",
    "ireland": "Ireland",
    "finland": "Finland",
    "switzerland": "Switzerland",
    "denmark": "Denmark",
    "sweden": "Sweden",
    "norway": "Norway",
    "spain": "Spain",
    "portugal": "Portugal",
    "poland": "Poland",
    "italy": "Italy",
    "austria": "Austria",
    "belgium": "Belgium",
    "croatia": "Croatia",
    "cyprus": "Cyprus",
    "estonia": "Estonia",
    "hungary": "Hungary",
    "latvia": "Latvia",
    "lithuania": "Lithuania",
    "luxembourg": "Luxembourg",
    "malta": "Malta",
    "romania": "Romania",
    "serbia": "Serbia",
    "slovakia": "Slovakia",
    "slovenia": "Slovenia",
    "ukraine": "Ukraine",
    "russia": "Russia",

    "argentina": "Argentina",
    "brazil": "Brazil",
    "chile": "Chile",
    "colombia": "Colombia",
    "costa rica": "Costa Rica",
    "ecuador": "Ecuador",
    "jamaica": "Jamaica",
    "mexico": "Mexico",
    "panama": "Panama",
    "panamá": "Panama",
    "trinidad and tobago": "Trinidad and Tobago",
    "uruguay": "Uruguay",

    "china": "China",
    "mainland china": "China",
    "india": "India",
    "indonesia": "Indonesia",
    "israel": "Israel",
    "japan": "Japan",
    "jordan": "Jordan",
    "kuwait": "Kuwait",
    "malaysia": "Malaysia",
    "pakistan": "Pakistan",
    "philippines": "Philippines",
    "qatar": "Qatar",
    "saudi arabia": "Saudi Arabia",
    "singapore": "Singapore",
    "south korea": "South Korea",
    "sri lanka": "Sri Lanka",
    "thailand": "Thailand",
    "vietnam": "Vietnam",

    "ghana": "Ghana",
    "kenya": "Kenya",
    "morocco": "Morocco",
    "nigeria": "Nigeria",
    "south africa": "South Africa",
    "tanzania": "Tanzania",
    "uganda": "Uganda",
    "zambia": "Zambia",
    "zimbabwe": "Zimbabwe",

    "united arab emirates": "United Arab Emirates",
    "saudi arabia": "Saudi Arabia",
    "qatar": "Qatar",
}

In [490]:
df["pais_clean"] = df["pais_key"].map(country_map)

df["pais_invalido"] = df["pais_clean"].isna() | df["pais_no_pais"]
print("Registros inválidos (se eliminarán):", df["pais_invalido"].sum())

df = df[~df["pais_invalido"]].copy()

Registros inválidos (se eliminarán): 162


In [491]:
df.loc[df["pais_clean"].isna(), "pais_raw"].value_counts().head(50)

,count
pais_raw,


In [492]:
print(" Numero de paises totales: " ,df["pais_clean"].nunique())
df["pais_clean"].unique()

 Numero de paises totales:  76


array(['United States', 'United Kingdom', 'Canada', 'Netherlands',
       'Australia', 'Finland', 'France', 'Germany', 'Ireland', 'India',
       'Argentina', 'Denmark', 'Spain', 'Switzerland', 'Malaysia',
       'Mexico', 'South Africa', 'Belgium', 'Sweden', 'Hong Kong',
       'Kuwait', 'Norway', 'Sri Lanka', 'Austria', 'Brazil', 'Hungary',
       'Luxembourg', 'Colombia', 'New Zealand', 'Trinidad and Tobago',
       'Czechia', 'Latvia', 'Puerto Rico', 'United Arab Emirates',
       'Romania', 'Serbia', 'Philippines', 'Russia', 'Poland', 'China',
       'Japan', 'Taiwan', 'Vietnam', 'Singapore', 'South Korea',
       'Thailand', 'Israel', 'Lithuania', 'Indonesia', 'Italy',
       'Slovenia', 'Slovakia', 'Portugal', 'Costa Rica', 'Chile', 'Qatar',
       'Nigeria', 'Panama', 'Uruguay', 'Pakistan', 'Uganda', 'Malta',
       'Saudi Arabia', 'Estonia', 'Morocco', 'Ecuador', 'Zimbabwe',
       'Ghana', 'Croatia', 'Ukraine', 'Jamaica', 'Kenya', 'Jordan',
       'Cyprus', 'Tanzania', 'Zambi

In [493]:
df = df.drop(columns=["pais_raw", "pais_key", "pais_no_pais", "pais_invalido"])
df = df.drop(columns=["pais"])
df = df.rename(columns={"pais_clean": "pais"})

In [494]:
capital_map = {
    "United States": "Washington, D.C.",
    "Canada": "Ottawa",
    "Mexico": "Mexico City",
    "Puerto Rico": "San Juan",
    "Trinidad and Tobago": "Port of Spain",
    "Jamaica": "Kingston",
    "Costa Rica": "San José",
    "Panama": "Panama City",
    "Argentina": "Buenos Aires",
    "Brazil": "Brasília",
    "Chile": "Santiago",
    "Colombia": "Bogotá",
    "Ecuador": "Quito",
    "Uruguay": "Montevideo",
    "United Kingdom": "London",
    "Netherlands": "Amsterdam",
    "Finland": "Helsinki",
    "France": "Paris",
    "Germany": "Berlin",
    "Ireland": "Dublin",
    "Denmark": "Copenhagen",
    "Spain": "Madrid",
    "Switzerland": "Bern",
    "Belgium": "Brussels",
    "Sweden": "Stockholm",
    "Norway": "Oslo",
    "Austria": "Vienna",
    "Hungary": "Budapest",
    "Luxembourg": "Luxembourg",
    "Czechia": "Prague",
    "Latvia": "Riga",
    "Romania": "Bucharest",
    "Serbia": "Belgrade",
    "Poland": "Warsaw",
    "Lithuania": "Vilnius",
    "Italy": "Rome",
    "Slovenia": "Ljubljana",
    "Slovakia": "Bratislava",
    "Portugal": "Lisbon",
    "Estonia": "Tallinn",
    "Croatia": "Zagreb",
    "Ukraine": "Kyiv",
    "Cyprus": "Nicosia",
    "Malta": "Valletta",
    "Russia": "Moscow",
    "India": "New Delhi",
    "Malaysia": "Kuala Lumpur",
    "Kuwait": "Kuwait City",
    "Sri Lanka": "Sri Jayawardenepura Kotte",
    "Philippines": "Manila",
    "China": "Beijing",
    "Japan": "Tokyo",
    "Taiwan": "Taipei",
    "Vietnam": "Hanoi",
    "Singapore": "Singapore",
    "South Korea": "Seoul",
    "Thailand": "Bangkok",
    "Israel": "Jerusalem",
    "Indonesia": "Jakarta",
    "Pakistan": "Islamabad",
    "Jordan": "Amman",
    "United Arab Emirates": "Abu Dhabi",
    "Qatar": "Doha",
    "Saudi Arabia": "Riyadh",
    "South Africa": "Pretoria",
    "Nigeria": "Abuja",
    "Uganda": "Kampala",
    "Morocco": "Rabat",
    "Zimbabwe": "Harare",
    "Ghana": "Accra",
    "Kenya": "Nairobi",
    "Tanzania": "Dodoma",
    "Zambia": "Lusaka",
    "Australia": "Canberra",
    "New Zealand": "Wellington",
    "Hong Kong": "Hong Kong",
}

In [495]:
import numpy as np
import pandas as pd

mask_ciudad_falta = df["ciudad"].isna() | (df["ciudad"].astype(str).str.strip() == "")

df["ciudad_clean"] = df["ciudad"].where(~mask_ciudad_falta, df["pais"].map(capital_map))

# Flag para documentar qué filas fueron imputadas
df["ciudad_imputada_capital"] = mask_ciudad_falta

In [496]:
df["ciudad_imputada_capital"].value_counts()

,count
ciudad_imputada_capital,
False,27379
True,9


In [497]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27388 entries, 0 to 28190
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   rango_edad                 27388 non-null  object 
 1   industria                  27388 non-null  object 
 2   nombre_trabajo             27388 non-null  object 
 3   moneda                     27388 non-null  object 
 4   otra_moneda                192 non-null    object 
 5   info_adicionales_ingresos  2967 non-null   object 
 6   estado_pais                22595 non-null  object 
 7   ciudad                     27388 non-null  object 
 8   experiencia_general        27388 non-null  object 
 9   experiencia_campo          27388 non-null  object 
 10  nivel_educacion            27388 non-null  object 
 11  genero                     27388 non-null  object 
 12  etnia                      27388 non-null  object 
 13  salario_anual_monto        27388 non-null  int64  


In [498]:
mask_outliers_salario = df["salario_usd"] > 800_000
mask_outliers_salario.sum()

np.int64(33)

In [499]:
df.loc[
    mask_outliers_salario,
    ["pais", "industria", "nombre_trabajo", "salario_usd"]
].sort_values("salario_usd", ascending=False).head(50)

,pais,industria,nombre_trabajo,salario_usd
28055,Canada,"Accounting, Banking & Finance",Investment Banking Analyst,4.440052e+09
3605,Colombia,Utilities & Telecommunications,Operations Manager,1.020000e+08
28113,Colombia,Health care,Analista de actuaria,5.220000e+07
26466,United States,Sales,Inside sales manager,5.000044e+06
28030,China,"Accounting, Banking & Finance",student,5.000000e+06
2124,United States,Art & Design,Owner and CEO,3.000000e+06
15509,Singapore,Computing or Tech,Product Manager,2.111538e+06
28084,Taiwan,Social networks,Content creator,2.000000e+06
5755,United States,Health care,Attending Physician (general internal medicine),1.900000e+06
6783,United States,Computing or Tech,Principal Software Engineer,1.650000e+06


In [502]:
mask_outliers_compensacion = df["compensacion_usd"] > 500_000
mask_outliers_compensacion.sum()


np.int64(23)

In [503]:
df.loc[
    mask_outliers_compensacion,
    ["pais", "industria", "nombre_trabajo", "compensacion_usd"]
].sort_values("compensacion_usd", ascending=False).head(50)

,pais,industria,nombre_trabajo,compensacion_usd
26935,United States,"Accounting, Banking & Finance",Managing Director,1500000.0
18279,United States,"Accounting, Banking & Finance",Security Engineer,1400000.0
18322,United States,"Accounting, Banking & Finance",Security Engineer,1400000.0
11997,United States,Computing or Tech,Product Manager,1200000.0
11978,United States,Computing or Tech,Product Manager,1200000.0
20420,United States,Computing or Tech,Senior staff software engineer,1000000.0
23736,United States,Computing or Tech,Senior Business Systems Analyst,1000000.0
15275,United States,Computing or Tech,UX Designer,900000.0
18590,United States,Computing or Tech,Principal Engineer,900000.0
18237,United States,Computing or Tech,Senior Engineering Manager,900000.0


In [525]:
p1_salario = df["salario_usd"].quantile(0.01)
p95_salario = df["salario_usd"].quantile(0.95)

print("P1:", p1_salario)
print("P95:", p95_salario)

P1: 16348.221600000003
P95: 179538.8999999992


In [526]:
p5_compensacion  = df["compensacion_usd"].quantile(0.05)
p95_compensacion = df["compensacion_usd"].quantile(0.95)

print("P5:", p5_compensacion)
print("P95:", p95_compensacion)

P5: 0.0
P95: 35000.0


In [527]:
df_trim = df[
    (df["salario_usd"] >= p1_salario) &
    (df["salario_usd"] <= p95_salario)
].copy()

In [528]:
df_trim = df_trim[
    (df_trim["compensacion_usd"] <= p95_compensacion)
].copy()

In [529]:
print("Antes:", len(df))
print("Después:", len(df_trim))
print("Eliminados:", len(df) - len(df_trim))

Antes: 27388
Después: 24963
Eliminados: 2425


In [530]:
df_trim["salario_usd"].describe()

,salario_usd
count,24963.000000
mean,77483.211320
std,33684.348713
min,16350.000000
25%,52000.000000
50%,70300.000000
75%,97000.000000
max,179301.000000


In [531]:
df_trim["compensacion_usd"].describe()

,compensacion_usd
count,24963.000000
mean,3403.690970
std,6334.937455
min,0.000000
25%,0.000000
50%,0.000000
75%,4187.500000
max,35000.000000


In [532]:
df_trim = df_trim.reset_index(drop=True)

df_trim.index = df_trim.index + 1
df_trim.index.name = "id"

df_trim.to_excel(
    "salary_survey_modelado.xlsx",
    index=False
)